In [1]:
import itertools

In [2]:
n = 330

map = [set(v) for v in itertools.combinations(range(1, 12), 4)] # map between vertices (integers between 0 and 329) and actual sets
v1 = map.index({1,2,3,4})
v2 = map.index({5,6,7,8})

G = [[False] * n for i in range(n)] # Kneser graph as an adjacency matrix
disjoint = [set() for i in range(n)] # for each vertex v, gives the neighborhood of v
intersecting = [set() for i in range(n)] # for each vertex v, gives list of vertices not in neighborhood of v

for i in range(n):
    for j in range(i + 1, n):
        if map[i].isdisjoint(map[j]):
            G[i][j] = True
            G[j][i] = True
            disjoint[i].add(j)
            disjoint[j].add(i)
        else:
            intersecting[i].add(j)
            intersecting[j].add(i)

In [3]:
print("|V(G)| = " + str(len(G)))
tot = 0
for i in range(n):
    tot += len(disjoint[i])
print("|E(G)| = " + str(tot // 2))
print("|d(G)| = " + str(len(disjoint[0])))

|V(G)| = 330
|E(G)| = 5775
|d(G)| = 35


In [4]:
def add_singleton(g):
    """
    Returns set of all possible singleton additions, where the singleton is connected to g.
    g must be a sorted tuple of vertices in G.
    g represents the graph of edges/singletons we're building up.
    """
    a = intersecting[g[0]]
    for i in range(1, len(g)):
        a &= intersecting[g[i]]
    return set(tuple(sorted(list(g) + [v])) for v in a)

def add_edge(g):
    """
    Returns set of all possible edge additions, where the edge is not connected to g.
    g must be a sorted tuple of vertices in G.
    g represents the graph of edges/singletons we're building up.
    """
    result = set()
    a = intersecting[g[0]]
    for i in range(1, len(g)):
        a &= intersecting[g[i]]
    for v in a:
        for w in a:
            if v < w and G[v][w]:
                result.add(tuple(sorted(list(g) + [v, w])))
    return result

In [5]:
print(len(add_singleton((v1,))))

294


In [6]:
edge_1 = {(v1, v2)} # possibilities with one edge

edge_2 = set() # possibilities with two edges
for h in add_edge((v1, v2)):
    # We can assume WLOG that one of the vertices of the second edge contains 1,5.
    flag = False
    for v in h:
        if 1 in map[v] and 5 in map[v]:
            flag = True
    if flag:
        edge_2.add(h)

print(len(edge_2))
tot = 0
for g in edge_2:
    if map.index({1,2,3,5}) in g:
        tot += 1
print(tot)

960
19


In [7]:
edge_3 = set() # possibilities with three edges
for g in edge_2:
    for h in add_edge(g):
        edge_3.add(h)

print(len(edge_3))

7516


In [8]:
def min_cutset_size(g):
    """
    Returns "minimum size cutset" must be if Kneser graph is disconnected to g.
    Calculates the union of all neighborhoods of vertices in g.
    g must be a tuple of vertices in G.
    """
    cutset = disjoint[g[0]]
    for i in range(1, len(g)):
        cutset = cutset.union(disjoint[g[i]])
    for v in g:
        if v in cutset:
            cutset.remove(v)
    return len(cutset)

In [9]:
min_c = n
for g in edge_3:
    min_c = min(min_c, min_cutset_size(g))
print(min_c)

131


In [12]:
edge_3_single_1 = set()
for g in edge_3:
    for h in add_singleton(g):
        edge_3_single_1.add(h)

print(len(edge_3_single_1))

min_c = n
for g in edge_3_single_1:
    min_c = min(min_c, min_cutset_size(g))
print(min_c)

0
330
